In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, permutations

#### Serial Episode - Parallel Episode - Non-Serial/Parallel Episode

**Serial Episode** : Comprises of a sequence of events that occur in a fixed order. An important note is that in order for an event to begin it's precedent event must be completed.

    Wake Up -> Brush Teeth -> Eat Breakfast

**Parallel Episode** : Comprises of a set of events that can occur in any order and/or simultaneously.

    Cut Vegetables -> Watch Recipe Video -> Salt Boiling Water

**Non-Serial Episode** : Comprises of a set of events that can occur in any order but **not** simultaneously
    
    Shop Groceries -> Work On Project -> Eat Dinner


#### MINEPI vs WINEPI

MINEPI (Minimum Description Length-based Episodic Pattern Mining) and WINEPI (Weighted Minimum Description Length-based Episodic Pattern Mining) are algorithms for mining episodic patterns from sequential data. 

Key differences:

1. MINEPI is based on the MDL (Minimum Description Principle) that finds patterns that can compress the data the most. Conversly, WINEPI is based on the weighted MDL principle that considers the compression of the data as much as the importance of the pattern.

2. MINEPI assumes that all episodes are unweighted and hence contribute equally to the sequence. However, WINEPI allows for weighted episodes where their importance is indicated by a weight.

3. MINEPI searches for patterns using BFS (Breadth-First Search) while WINEPI using beam search. Therefore, WINEPI may be more computationally efficient but some patterns may only be discovered using MINEPI.

4. MINEPI is more robust to data noise since it adopts a threshold based approach to filter out statistically insignificant patterns. Inversly, due to the fact that WINEPI assigns weights to the episodes, it is more sensitive to outliers. 


Let's load our data into a DataFrame object

In [2]:
df_loan = pd.read_csv('Artificial - Loan Process.csv')
df_loan

,case,event,completeTime
0,trace 0,register application,2013-04-16 10:08:02
1,trace 0,check credit,2013-04-16 10:16:03
2,trace 0,calculate capacity,2013-04-16 10:16:28
3,trace 0,check system,2013-04-16 10:20:25
4,trace 0,accept,2013-04-16 10:21:30
...,...,...,...
585,trace 99,register application,2013-04-16 18:30:02
586,trace 99,calculate capacity,2013-04-16 18:39:07
587,trace 99,check credit,2013-04-16 18:48:13
588,trace 99,reject,2013-04-16 18:50:59


We detect that the *completeTime* column contains timestamps so it is natural to check if it is of the correct data type.

In [3]:
df_loan.dtypes

case            object
event           object
completeTime    object
dtype: object

We detect that it's a string so we will convert it to a time series and use it to create another column of unix timestamps. We do this to preserve the timestamp integrity but convert it to a format than can be processed and compared to.

In [4]:
df_loan['completeTime_unix'] = pd.to_datetime(df_loan['completeTime']).apply(lambda x: int(x.timestamp()))
df_loan

,case,event,completeTime,completeTime_unix
0,trace 0,register application,2013-04-16 10:08:02,1366106882
1,trace 0,check credit,2013-04-16 10:16:03,1366107363
2,trace 0,calculate capacity,2013-04-16 10:16:28,1366107388
3,trace 0,check system,2013-04-16 10:20:25,1366107625
4,trace 0,accept,2013-04-16 10:21:30,1366107690
...,...,...,...,...
585,trace 99,register application,2013-04-16 18:30:02,1366137002
586,trace 99,calculate capacity,2013-04-16 18:39:07,1366137547
587,trace 99,check credit,2013-04-16 18:48:13,1366138093
588,trace 99,reject,2013-04-16 18:50:59,1366138259


We sort our episodes based on their timestamps.

In [5]:
df_loan.sort_values(by='completeTime', ascending=True, inplace=True)
df_loan.reset_index(inplace=True)
df_loan.drop(columns='index', inplace=True)
df_loan

,case,event,completeTime,completeTime_unix
0,trace 0,register application,2013-04-16 10:08:02,1366106882
1,trace 1,register application,2013-04-16 10:10:02,1366107002
2,trace 2,register application,2013-04-16 10:15:42,1366107342
3,trace 0,check credit,2013-04-16 10:16:03,1366107363
4,trace 1,check credit,2013-04-16 10:16:07,1366107367
...,...,...,...,...
585,trace 99,check credit,2013-04-16 18:48:13,1366138093
586,trace 98,send decision e-mail,2013-04-16 18:50:47,1366138247
587,trace 99,reject,2013-04-16 18:50:59,1366138259
588,trace 97,send decision e-mail,2013-04-16 18:51:36,1366138296


We assign each different event to an integer for comparability purposes.

In [6]:
event_dict = {'register application' : '1', 'check credit' : '2', 'calculate capacity' : '3', 
              'check system' : '4', 'accept' : '5', 'send decision e-mail' : '6', 'reject' : '7'}

for index in df_loan.index:
    df_loan.at[index, 'event_num'] = event_dict[df_loan.at[index, 'event']] 
df_loan

,case,event,completeTime,completeTime_unix,event_num
0,trace 0,register application,2013-04-16 10:08:02,1366106882,1
1,trace 1,register application,2013-04-16 10:10:02,1366107002,1
2,trace 2,register application,2013-04-16 10:15:42,1366107342,1
3,trace 0,check credit,2013-04-16 10:16:03,1366107363,2
4,trace 1,check credit,2013-04-16 10:16:07,1366107367,2
...,...,...,...,...,...
585,trace 99,check credit,2013-04-16 18:48:13,1366138093,2
586,trace 98,send decision e-mail,2013-04-16 18:50:47,1366138247,6
587,trace 99,reject,2013-04-16 18:50:59,1366138259,7
588,trace 97,send decision e-mail,2013-04-16 18:51:36,1366138296,6


We create a list of tuples in the format of (timestamp, event)

In [7]:
episodes = (df_loan[['completeTime_unix', 'event_num']].to_records(index=False)).tolist()
episodes

[(1366106882, '1'),
 (1366107002, '1'),
 (1366107342, '1'),
 (1366107363, '2'),
 (1366107367, '2'),
 (1366107381, '3'),
 (1366107388, '3'),
 (1366107596, '1'),
 (1366107611, '4'),
 (1366107625, '4'),
 (1366107690, '5'),
 (1366107761, '2'),
 (1366107787, '2'),
 (1366107817, '3'),
 (1366107856, '5'),
 (1366108013, '6'),
 (1366108121, '3'),
 (1366108123, '1'),
 (1366108158, '4'),
 (1366108198, '6'),
 (1366108328, '2'),
 (1366108408, '1'),
 (1366108413, '4'),
 (1366108507, '1'),
 (1366108589, '2'),
 (1366108631, '3'),
 (1366108674, '5'),
 (1366108706, '1'),
 (1366108732, '3'),
 (1366108847, '4'),
 (1366108857, '2'),
 (1366108931, '2'),
 (1366108967, '3'),
 (1366108971, '5'),
 (1366108975, '3'),
 (1366109038, '4'),
 (1366109043, '6'),
 (1366109066, '5'),
 (1366109109, '6'),
 (1366109133, '5'),
 (1366109193, '4'),
 (1366109243, '1'),
 (1366109249, '1'),
 (1366109293, '6'),
 (1366109372, '1'),
 (1366109447, '6'),
 (1366109466, '1'),
 (1366109469, '7'),
 (1366109486, '2'),
 (1366109493, '4'),


We initialize a WINEPI object. And specify that we are looking for parallel episodes.

In [8]:
from episode_mining.winepi import WINEPI, WinEpiRules

winepi_miner = WINEPI(sequence=episodes, episode_type='parallel')

We set the minimum support threshold to 0.001

In [32]:
freqItems, suppData = winepi_miner.WinEpi(width=10, step=4, minFrequent=0.001)

In [33]:
winepiRules = WinEpiRules(freqItems, suppData, width=4, minConfidence=0.05)
ruleList = winepiRules.generateRules()
winepiRules.printRules(ruleList)

WINEPIRule: ['7'] ==> ['4', '7'] [4] [0.0012634238787113076, 0.05128205128205128]
WINEPIRule: ['7'] ==> ['1', '7'] [4] [0.0013897662665824384, 0.05641025641025641]


We experiment with the hyperparameter values and the results they produce.

In [17]:
freqItems, suppData = winepi_miner.WinEpi(width=10, step=4, minFrequent=0.0006)

In [29]:
winepiRules = WinEpiRules(freqItems, suppData, width=4, minConfidence=0.04)
ruleList = winepiRules.generateRules()
winepiRules.printRules(ruleList)

WINEPIRule: ['1'] ==> ['1', '2'] [4] [0.0013897662665824384, 0.045081967213114756]
WINEPIRule: ['2'] ==> ['1', '2'] [4] [0.0013897662665824384, 0.04526748971193415]
WINEPIRule: ['4'] ==> ['4', '7'] [4] [0.0012634238787113076, 0.04444444444444444]
WINEPIRule: ['7'] ==> ['4', '7'] [4] [0.0012634238787113076, 0.05128205128205128]
WINEPIRule: ['1'] ==> ['1', '7'] [4] [0.0013897662665824384, 0.045081967213114756]
WINEPIRule: ['7'] ==> ['1', '7'] [4] [0.0013897662665824384, 0.05641025641025641]
WINEPIRule: ['4'] ==> ['4', '6'] [4] [0.001137081490840177, 0.04]
WINEPIRule: ['1'] ==> ['1', '4'] [4] [0.0013897662665824384, 0.045081967213114756]
WINEPIRule: ['4'] ==> ['1', '4'] [4] [0.0013897662665824384, 0.048888888888888885]
WINEPIRule: ['1'] ==> ['1', '3'] [4] [0.0013897662665824384, 0.045081967213114756]
WINEPIRule: ['3'] ==> ['1', '3'] [4] [0.0013897662665824384, 0.04453441295546559]


In [34]:
freqItems, suppData = winepi_miner.WinEpi(width=10, step=4, minFrequent=0.0006)

In [36]:
winepiRules = WinEpiRules(freqItems, suppData, width=4, minConfidence=0.01)
ruleList = winepiRules.generateRules()
winepiRules.printRules(ruleList)

WINEPIRule: ['3'] ==> ['3', '7'] [4] [0.0006317119393556538, 0.020242914979757085]
WINEPIRule: ['7'] ==> ['3', '7'] [4] [0.0006317119393556538, 0.02564102564102564]
WINEPIRule: ['2'] ==> ['2', '3'] [4] [0.0007580543272267846, 0.024691358024691357]
WINEPIRule: ['3'] ==> ['2', '3'] [4] [0.0007580543272267846, 0.024291497975708502]
WINEPIRule: ['3'] ==> ['3', '4'] [4] [0.0008843967150979153, 0.02834008097165992]
WINEPIRule: ['4'] ==> ['3', '4'] [4] [0.0008843967150979153, 0.03111111111111111]
WINEPIRule: ['1'] ==> ['1', '2'] [4] [0.0013897662665824384, 0.045081967213114756]
WINEPIRule: ['2'] ==> ['1', '2'] [4] [0.0013897662665824384, 0.04526748971193415]
WINEPIRule: ['2'] ==> ['2', '6'] [4] [0.0008843967150979153, 0.028806584362139915]
WINEPIRule: ['6'] ==> ['2', '6'] [4] [0.0008843967150979153, 0.028455284552845527]
WINEPIRule: ['3'] ==> ['3', '6'] [4] [0.0008843967150979153, 0.02834008097165992]
WINEPIRule: ['6'] ==> ['3', '6'] [4] [0.0008843967150979153, 0.028455284552845527]
WINEPIRul

Now we will implement MINEPI.

In [14]:
def createC1(sequence, step):
    C1 = {}
    for event in sequence:
        mo = tuple((event[0], event[0] + step)) # calc minimal occurrence (mo)
        evt1 = tuple(event[1])
        if evt1 in C1: # if event exists in C1
            if mo not in C1[evt1]: # if mo not exists in event's set of mo
                C1[evt1].add(mo)
        else: C1[evt1] = set((mo,))
    return C1

In [15]:
def createLargeEpisodeSet(Ck, min_fr):
    Lk = Ck.copy()
    for episode in Ck:
        if len(Ck[episode]) < min_fr: # if number of mo < min_fr
            del Lk[episode]
    return Lk

In [16]:
def checkSubsetFrequency(candidate, Lk, k):
    if k > 1:    
        subsets = list(combinations(candidate, k))
    else:
        return True
    for elem in subsets:
        if not elem in Lk: #elem is tuple
            return False
    return True

In [17]:
def createCandidates(Lk, L1, k, max_width):
    # create candidates list
    candidatesK = [] #candidates list
    generatedCans = [] 
    lkKeys = sorted(set([item for t in Lk for item in t])) # get all episodes and sort / Lk:get all keys
    generatedCans = list(combinations(lkKeys, k))
    for can in generatedCans:
        if checkSubsetFrequency(can, Lk, k-1): #if all subsets of can is frequent / Lk:get all keys
            candidatesK.append(can)
    # create minimal occurrence of candidates
    if len(candidatesK) == 0: # if candidates list is null
        return dict()
    else:
        Ck = {}
        if k == 2: #candidate 2
            for can in candidatesK: # can is tuple
                Ck[can] = set()
                for mok in Lk[tuple(can[0])]:
                    for mo1 in L1[tuple(can[1])]:
                        tsmin = min(mok[0], mo1[0])
                        temax = max(mok[1], mo1[1])
                        # temax - tsmin <= max_width
                        if temax - tsmin <= max_width:
                            Ck[can].add(tuple((tsmin, temax)))
        else: #candidate > 2
            for can in candidatesK: # can is tuple
                Ck[can] = set()
                for mok in Lk[can[:-1]]:
                    for mo1 in L1[tuple(can[-1])]:
                        tsmin = min(mok[0], mo1[0])
                        temax = max(mok[1], mo1[1])
                        # temax - tsmin <= max_width
                        if temax - tsmin <= max_width:
                            Ck[can].add(tuple((tsmin, temax)))
        # recheck minimal occurrence is minimal
        for episode, setOfMOs in Ck.items():
            rem = set() #remove set
            for mo1 in setOfMOs:
                for mo2 in setOfMOs.difference(mo1):
                    # mo1[0] is mo1.ts, mo2[0] is mo2.ts, mo1[1] is mo1.te, mo2[1] is mo2.te
                    if (mo1[0]==mo2[0] and mo1[1] < mo2[1]) or (mo1[0] > mo2[0] and mo1[1] == mo2[1]) or (mo1[0] > mo2[0] and mo1[1] < mo2[1]):
                        rem.add(mo2)
            Ck[episode] = setOfMOs.difference(rem)
        return Ck

In [18]:
def MinEpi(sequence, max_width, step, minFrequent):
    C1 = createC1(sequence, step)
    L1 = createLargeEpisodeSet(C1, minFrequent)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = createCandidates(L[k-2], L1, k, max_width)
        Lk = createLargeEpisodeSet(Ck, minFrequent)
        L.append(Lk)
        k += 1
    #remove empty last itemset from L
    if L[-1] == {}:
        L.pop()
    return L

L = MinEpi(episodes, max_width=5, step=1, minFrequent=2)
L

[{('1',): {(1366106882, 1366106883),
   (1366107002, 1366107003),
   (1366107342, 1366107343),
   (1366107596, 1366107597),
   (1366108123, 1366108124),
   (1366108408, 1366108409),
   (1366108507, 1366108508),
   (1366108706, 1366108707),
   (1366109243, 1366109244),
   (1366109249, 1366109250),
   (1366109372, 1366109373),
   (1366109466, 1366109467),
   (1366109665, 1366109666),
   (1366110134, 1366110135),
   (1366110327, 1366110328),
   (1366110577, 1366110578),
   (1366111051, 1366111052),
   (1366111153, 1366111154),
   (1366111530, 1366111531),
   (1366112046, 1366112047),
   (1366112350, 1366112351),
   (1366112494, 1366112495),
   (1366112634, 1366112635),
   (1366113232, 1366113233),
   (1366113383, 1366113384),
   (1366113534, 1366113535),
   (1366114103, 1366114104),
   (1366114502, 1366114503),
   (1366114941, 1366114942),
   (1366114998, 1366114999),
   (1366115039, 1366115040),
   (1366115496, 1366115497),
   (1366115623, 1366115624),
   (1366115988, 1366115989),
   (13

In [19]:
def isAlphaOccurInBeta(mob, mos_alpha): #mo_beta include (mo_beta_ts) and (mo_beta_ts + width2)
    for moa in mos_alpha:
        if mob[0] <= moa[0] and moa[1] <= mob[1]:
            return True
    return False

In [20]:
def calcConfidence(mos_beta, wid1, mos_alpha, wid2):
    denominator = 0.0
    numerator = 0
    for mob in mos_beta:
        #te - ts <= width1
        if mob[1] - mob[0] <= wid1:
            denominator += 1
            #number of mo_beta [mob.ts, mob.ts + wid2) contains mo_alpha is numerator
            if isAlphaOccurInBeta((mob[0], mob[0] + wid2), mos_alpha):
                numerator += 1
    if denominator == 0:
        return 0
    else:  
        return numerator/denominator

In [21]:
def calcSupport(mos_alpha, wid2):
    support = 0
    for moa in mos_alpha:
        #te - ts <= width2
        if moa[1] - moa[0] <= wid2:
            support += 1
    return support

In [22]:
def generateRules(L, max_width, step, minConf):
        #calculate time bound W
        time_bound = list(range(step, max_width, step))
        time_bound.append(max_width)
        #generate all rules\n",
        for i in range(1, len(L)): #only get the sets with two or more items\n",
            for episode in L[i]: #for each episode in a level\n",
                for j in range(1, i+1): # i+1 equal to length of an episode, so j is the length of lhs\n",
                    lhsList = list(combinations(episode, j))
                    for lhs in lhsList: #each left-hand-side\n",
                        for rightWid in time_bound: #each time bound of rhs\n",
                            for leftWid in time_bound: #each time bound of lhs\n",
                                #index of L is the episode length-1\n",
                                conf = calcConfidence(L[j-1][lhs], leftWid, L[i][episode], rightWid)
                                if conf >= minConf:
                                    event1 = ([key for key in event_dict.keys() if event_dict[key] == list(lhs)[0]])
                                    events2 = [key for key, value in event_dict.items() if value in list(episode)]
                                    print(event1,f' [{leftWid}] ==> {events2}, [{rightWid}] [{calcSupport(L[i][episode], rightWid)}, {conf}]')

                                    

In [23]:
generateRules(L, 5, 1, 0.005)

['register application']  [1] ==> ['register application', 'check credit'], [3] [3, 0.01]
['register application']  [2] ==> ['register application', 'check credit'], [3] [3, 0.01]
['register application']  [3] ==> ['register application', 'check credit'], [3] [3, 0.01]
['register application']  [4] ==> ['register application', 'check credit'], [3] [3, 0.01]
['register application']  [5] ==> ['register application', 'check credit'], [3] [3, 0.01]
['register application']  [1] ==> ['register application', 'check credit'], [4] [3, 0.01]
['register application']  [2] ==> ['register application', 'check credit'], [4] [3, 0.01]
['register application']  [3] ==> ['register application', 'check credit'], [4] [3, 0.01]
['register application']  [4] ==> ['register application', 'check credit'], [4] [3, 0.01]
['register application']  [5] ==> ['register application', 'check credit'], [4] [3, 0.01]
['register application']  [1] ==> ['register application', 'check credit'], [5] [4, 0.01]
['register